# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os
import time

import gmaps.datasets

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
# Loading csv file exported from WeatherPy to new DataFrame:
path = "../output_data/clean_city_data.csv"

# Reading in the csv file and creating the new dataframe:
new_city_df = pd.read_csv(path)

# Cleaning new cihty dataframe:
city_df_cleaned = new_city_df.dropna()
city_df_cleaned


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Saskylakh,71.92,114.08,1.72,92,98,6.87,RU,1603815811
1,Adrar,20.50,-10.07,96.51,10,0,10.25,MR,1603815798
2,Lorengau,-2.02,147.27,80.35,76,79,1.68,PG,1603816033
3,Zhigansk,66.77,123.37,-4.40,92,90,2.80,RU,1603815982
4,Talara,-4.58,-81.27,73.40,69,0,17.22,PE,1603816035
...,...,...,...,...,...,...,...,...,...
550,Newport,51.59,-3.00,55.00,82,64,3.00,GB,1603816387
551,Santa Inês,-3.67,-45.38,95.34,42,100,6.04,BR,1603816675
552,Truckee,39.33,-120.18,44.01,45,5,9.26,US,1603816676
553,Flinders,-34.58,150.86,55.99,89,47,1.01,AU,1603816677


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps:
gmaps.configure(api_key=g_key)

In [6]:
# Using Lat and Lng as locations:
locations = city_df_cleaned[["Lat", "Lng"]]

# Using Humidity as the weight: 
humidity = city_df_cleaned["Humidity"].astype(float)

In [7]:
# Add Heatmap layer to map:
fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heatmap_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Narrow down the cities to fit weather conditions"

# Zero cloudiness:
cloud_df = city_df_cleaned[city_df_cleaned["Cloudiness"] == 0]

# A max temperature lower than 80 degrees but higher than 70:
temp_df = cloud_df[(cloud_df["Max Temp"] > 70) & (cloud_df["Max Temp"] < 80)]

# Wind speed less than 10 mph:
wind_df = temp_df[temp_df["Wind Speed"] < 10]

# Drop any rows will null values:
wind_df = wind_df.dropna()
wind_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
88,Tezu,27.92,96.17,72.93,64,0,3.38,IN,1603816132
107,Gweru,-19.45,29.82,77.68,18,0,7.45,ZW,1603816153
112,Sur,22.57,59.53,72.19,56,0,1.25,OM,1603816159
124,Damietta,31.42,31.81,78.80,78,0,8.05,EG,1603816173
140,Erzin,36.96,36.20,77.00,22,0,5.82,TR,1603816191
262,Tikrit,34.61,43.68,74.44,18,0,9.13,IQ,1603816336
267,Mutoko,-17.40,32.23,75.67,27,0,9.98,ZW,1603816342
453,Bilma,18.69,12.92,77.52,17,0,3.33,NE,1603816559
460,Baghdad,33.34,44.40,73.40,28,0,3.36,IQ,1603816567
461,Haimen,23.19,116.61,71.60,78,0,4.47,CN,1603816569


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# Store into variable named hotel_df:
hotel_df = wind_df

# Add a "Hotel Name" column to the DataFrame: (setting new columns to hold values)
hotel_df["Hotel Name"] = ""

# Set parameters to search for hotels with 5000 meters:

# Set params:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Loop through to identify hotels:
for index, row in hotel_df.iterrows():
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params['location'] = f'{lat},{lng}'
    
    hotel_data = requests.get(base_url, params=params).json()
    
    try:
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data['results'][0]['name']
        
    except: 
        print('Issues with finding a hotel at', row['City'])
        hotel_df.loc[index, "Hotel Name"] = "NA"
    
hotel_df

Issues with finding a hotel at Bilma
Issues with finding a hotel at Haimen


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
88,Tezu,27.92,96.17,72.93,64,0,3.38,IN,1603816132,Capital O 66991 Hotel Highway
107,Gweru,-19.45,29.82,77.68,18,0,7.45,ZW,1603816153,Regency Hotel Fairmile
112,Sur,22.57,59.53,72.19,56,0,1.25,OM,1603816159,Sur Plaza Hotel
124,Damietta,31.42,31.81,78.80,78,0,8.05,EG,1603816173,فندق المنشي دمياط
140,Erzin,36.96,36.20,77.00,22,0,5.82,TR,1603816191,Hattusa Vacation Thermal Club Erzin
262,Tikrit,34.61,43.68,74.44,18,0,9.13,IQ,1603816336,الاقسام الداخلية لكلية الطب
267,Mutoko,-17.40,32.23,75.67,27,0,9.98,ZW,1603816342,Mutoko Hotel
453,Bilma,18.69,12.92,77.52,17,0,3.33,NE,1603816559,NA
460,Baghdad,33.34,44.40,73.40,28,0,3.36,IQ,1603816567,Baghdad Hotel
461,Haimen,23.19,116.61,71.60,78,0,4.47,CN,1603816569,NA


In [10]:
#deletes any rows without hotels in 5000 meters
hotel_df = hotel_df[hotel_df["Hotel Name"] != "NA"]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
88,Tezu,27.92,96.17,72.93,64,0,3.38,IN,1603816132,Capital O 66991 Hotel Highway
107,Gweru,-19.45,29.82,77.68,18,0,7.45,ZW,1603816153,Regency Hotel Fairmile
112,Sur,22.57,59.53,72.19,56,0,1.25,OM,1603816159,Sur Plaza Hotel
124,Damietta,31.42,31.81,78.80,78,0,8.05,EG,1603816173,فندق المنشي دمياط
140,Erzin,36.96,36.20,77.00,22,0,5.82,TR,1603816191,Hattusa Vacation Thermal Club Erzin
262,Tikrit,34.61,43.68,74.44,18,0,9.13,IQ,1603816336,الاقسام الداخلية لكلية الطب
267,Mutoko,-17.40,32.23,75.67,27,0,9.98,ZW,1603816342,Mutoko Hotel
460,Baghdad,33.34,44.40,73.40,28,0,3.36,IQ,1603816567,Baghdad Hotel
538,Kiryat Gat,31.61,34.76,79.00,100,0,3.00,IL,1603816660,Desert Gat


In [11]:
# Checking hotels:
len(hotel_df[["Lat", "Lng"]])

9

In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Checking hotel info:
len(hotel_info)

9

In [14]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))